# Create Heatmaps for Weather Parameters

## Import API and Dependencies 

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key
gmaps.configure(api_key=g_key)

## Create DataFrame from saved CSV

In [2]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Opuwo,NaN,2021-11-30 02:19:25,-18.0607,13.8400,67.51,30,71,1.72
1,1,Nome,US,2021-11-30 02:19:26,64.5011,-165.4064,-7.53,59,1,5.75
2,2,Bluff,NZ,2021-11-30 02:19:26,-46.6000,168.3333,62.80,75,92,6.04
3,3,Cape Town,ZA,2021-11-30 02:19:27,-33.9258,18.4232,63.09,84,90,6.91
4,4,Saint George,US,2021-11-30 02:19:27,37.1041,-113.5841,57.79,34,1,0.00


In [3]:
# identify the types of data in the DataFrame
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

## Create a Maximum Temperature Heatmap

In [4]:
# Assign the locations to an array of latitude and longitude pairs
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Humidity Heatmap

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Cloudiness Heatmap

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Wind Speed Heatmap

In [7]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Get Vacation Criteria

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Nabire,ID,2021-11-30 02:15:55,-3.3667,135.4833,88.48,60,97,2.35
8,8,Rikitea,PF,2021-11-30 02:15:09,-23.1203,-134.9692,76.44,78,5,20.33
9,9,Kapaa,US,2021-11-30 02:17:06,22.0752,-159.3190,80.58,75,90,5.01
12,12,Belmonte,BR,2021-11-30 02:19:31,-15.8631,-38.8828,77.50,92,99,11.77
13,13,Alyangula,AU,2021-11-30 02:19:32,-13.8483,136.4192,89.02,57,11,3.74
14,14,Avarua,CK,2021-11-30 02:15:15,-21.2078,-159.7750,80.65,57,20,14.97
17,17,Salinopolis,BR,2021-11-30 02:17:30,-0.6136,-47.3561,79.45,86,38,9.98
19,19,Auki,SB,2021-11-30 02:19:34,-8.7676,160.7034,85.17,70,69,2.44
25,25,Kahului,US,2021-11-30 02:19:37,20.8947,-156.4700,78.82,75,20,16.11
31,31,New Norfolk,AU,2021-11-30 02:17:11,-42.7826,147.0587,79.66,28,5,1.99


In [10]:
preferred_cities_df.count()

City_ID       153
City          153
Country       153
Date          153
Lat           153
Lng           153
Max Temp      153
Humidity      153
Cloudiness    153
Wind Speed    153
dtype: int64

## Map Vacation Criteria

### Get Travel Destinations

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Nabire,ID,88.48,-3.3667,135.4833,
8,Rikitea,PF,76.44,-23.1203,-134.9692,
9,Kapaa,US,80.58,22.0752,-159.3190,
12,Belmonte,BR,77.50,-15.8631,-38.8828,
13,Alyangula,AU,89.02,-13.8483,136.4192,
14,Avarua,CK,80.65,-21.2078,-159.7750,
17,Salinopolis,BR,79.45,-0.6136,-47.3561,
19,Auki,SB,85.17,-8.7676,160.7034,
25,Kahului,US,78.82,20.8947,-156.4700,
31,New Norfolk,AU,79.66,-42.7826,147.0587,


## Retrieve Hotels from a Nearby Search

In [12]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Nabire,ID,88.48,-3.3667,135.4833,Hotel Nusantara Nabire
8,Rikitea,PF,76.44,-23.1203,-134.9692,People ThankYou
9,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Belmonte,BR,77.50,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
13,Alyangula,AU,89.02,-13.8483,136.4192,Groote Eylandt Lodge
...,...,...,...,...,...,...
549,San Policarpo,PH,84.38,12.1791,125.5072,Agapita M. Nicart
556,Catuday,PH,82.47,16.2923,119.8062,Benjie's Kubo & Cottages
557,Wad Rawah,SD,77.52,15.1603,33.1397,ود الترابي
558,Labuhan,ID,85.10,-6.8844,112.2051,PT PPI Lamongan Tambak C


## Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add template for pop up info window
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))